<a href="https://colab.research.google.com/github/pranitha2144/Internship-iith/blob/master/Multilabelcnnpractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

2.2.0


In [0]:
training_set = pd.read_csv("/content/drive/My Drive/catsanddogsdataset/train.csv")

training_imgs = ["{}.jpg".format(x) for x in list(training_set.id)]

training_labels_1 = list(training_set['class_name'])
training_labels_2 = list(training_set['breed'])
training_set = pd.DataFrame( {'Images': training_imgs,'Animal': training_labels_1, 'Breed' : training_labels_2})

#Changing the type  to str
training_set.Animal = training_set.Animal.astype(str)
training_set.Breed = training_set.Breed.astype(str)

In [0]:
training_set['New_class'] = training_set['Animal'] + training_set['Breed']

In [19]:
print(training_set.head())

         Images Animal Breed New_class
0  img_1313.jpg      1    11       111
1  img_0642.jpg      2    21       221
2  img_1450.jpg      1    11       111
3  img_2201.jpg      1    12       112
4  img_0964.jpg      2    21       221


In [20]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="/content/drive/My Drive/catsanddogsdataset/images_train",x_col="Images",
                                        y_col="New_class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=32)

Found 6206 validated image filenames belonging to 10 classes.


In [0]:
classifier = Sequential()

In [0]:
#First Convolutional layer
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))

In [0]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [0]:
#second Convolutional layer
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

In [0]:
#Flattening
classifier.add(Flatten())

In [0]:
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))

#Output Layer
classifier.add(Dense(units = 10 , activation = 'softmax'))

In [0]:
classifier.compile(optimizer = 'SGD', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])



In [28]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 126, 126, 56)      1568      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 63, 63, 56)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 61, 32)        16160     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 28800)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                1843264   
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

In [29]:
classifier.fit(train_generator, epochs = 64, steps_per_epoch = 60 )

Epoch 1/64
60/60 [==============================] - 52s 874ms/step - loss: 2.3027 - categorical_accuracy: 0.1052 - accuracy: 0.1052
Epoch 2/64
60/60 [==============================] - 52s 873ms/step - loss: 2.2941 - categorical_accuracy: 0.1224 - accuracy: 0.1224
Epoch 3/64
60/60 [==============================] - 52s 872ms/step - loss: 2.2831 - categorical_accuracy: 0.1349 - accuracy: 0.1349
Epoch 4/64
60/60 [==============================] - 52s 875ms/step - loss: 2.2535 - categorical_accuracy: 0.1604 - accuracy: 0.1604
Epoch 5/64
60/60 [==============================] - 52s 871ms/step - loss: 2.2103 - categorical_accuracy: 0.1859 - accuracy: 0.1859
Epoch 6/64
60/60 [==============================] - 52s 873ms/step - loss: 2.1573 - categorical_accuracy: 0.2143 - accuracy: 0.2143
Epoch 7/64
60/60 [==============================] - 53s 878ms/step - loss: 2.1089 - categorical_accuracy: 0.2344 - accuracy: 0.2344
Epoch 8/64
60/60 [==============================] - 53s 882ms/step - loss: 2